# 10. Writing mapped trajectories from Gromacs all atom (AA) trajectories

### Prerequisites
* MDAnalysis
* gsd

This example maps a system of two 12 amino acid long peptides. One amino acid residue's center of mass represents one CG bead. The all atom topology and trajecotry files can be found in `examples` directory. The mapped trajectory will be saved in gsd format to `CG_tutorial` directory. 

In [1]:
import hoomd
import numpy as np 
import hoomd.md
import gsd,gsd.hoomd
import gsd.pygsd
import hoomd.htf as htf
import tensorflow as tf
import MDAnalysis as mda

# disable GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#disable warnings
import warnings
warnings.filterwarnings('ignore')

2 Physical GPUs, 2 Logical GPUs


Now let's read the trajectory and generate CG bead types, indices, COMs and total masses. There are 24 residues in the system. Hence we will name each bead index by an integer scaling from 1 to 24. CG beads names will be `B + index`. A user can give any bead name of choice. 

In [2]:
u = mda.Universe('test_topol.pdb','test_traj.trr')

In [3]:
#Here we will CG the atoms by their residues
N = len(u.atoms.residues)
print(N)
typeids = np.arange(1,N+1)

#creating bead names
types = []
for i in typeids:
    types.append('B'+str(i))
        
print('CG bead ids: ', typeids)
print('CG bead names: ', types)

# Now let's calculate the COMs and total masses of the CG groups
masses = []
mapped_pos = []

for ids in typeids:
    ag = u.select_atoms('resid '+str(ids))
    coms = ag.center_of_mass()
    masses.append(ag.total_mass())
    mapped_pos.append(coms)
    

24
CG bead ids:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
CG bead names:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24']


Here we generate snapshots of each state which will then be written to a GSD file. This will be called for all frames of the trajectory. But a user can specify the length of the trajectory by setting the `splice_traj=True` and by giving `traj_frames=[j,k]`. Here j is the first frame number and j is the last frame number you'd like to splice. For example to get the frame numbers between 10 and 35; `traj_frames=[10,35]`.

In [5]:
#Let's generate GSD file
# Note that this outfile will be written t
htf.CGmap_trajectory(N=N, CGtypes=types, CGids=typeids, mapped_positions=mapped_pos, 
                     CGmasses=masses, outfile='CG_tutorial/testtraj_mapped.gsd', mda_universe=u, splice_traj=False)

GSD file written
